In [1]:
import tensorflow as tf

In [2]:
#creating computation graph

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [5]:
# actual intialization, evaluation

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

In [6]:
print(result)

42


In [7]:
sess.close()

In [9]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [10]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [11]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [12]:
x.graph is tf.get_default_graph()

True

In [13]:
y.graph is tf.get_default_graph()

True

In [14]:
f.graph is tf.get_default_graph()

True

In [26]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    g = x2.graph is tf.get_default_graph()

In [27]:
g

True

In [23]:
x2.graph is graph

True

In [28]:
x2.graph is tf.get_default_graph()

False

In [30]:
# linear regression using hte normal function in TF

import tensorflow as tf
from sklearn.datasets import fetch_california_housing
import numpy as np

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [35]:
housing.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [37]:
housing.target.reshape(-1,1)

array([[4.526],
       [3.585],
       [3.521],
       ...,
       [0.923],
       [0.847],
       [0.894]])

In [39]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [40]:
with tf.Session() as sess:
    theta_Value = theta.eval()

In [41]:
theta_Value

array([[-3.7465141e+01],
       [ 4.3573415e-01],
       [ 9.3382923e-03],
       [-1.0662201e-01],
       [ 6.4410698e-01],
       [-4.2513184e-06],
       [-3.7732250e-03],
       [-4.2664889e-01],
       [-4.4051403e-01]], dtype=float32)

In [49]:
# using Batch gradient descent

# scale and nomalize the data first
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
housing_data_scales = scaler.fit_transform(housing.data)
#add bias 
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), housing_data_scales]

In [50]:
scaled_housing_data_plus_bias

array([[ 1.        ,  2.34476576,  0.98214266, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.        ,  1.7826994 ,  1.85618152, ..., -0.02584253,
         1.03850269, -1.33282653],
       ...,
       [ 1.        , -1.14259331, -0.92485123, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [ 1.        , -1.05458292, -0.84539315, ..., -0.09122515,
         1.77823747, -0.87362627],
       [ 1.        , -0.78012947, -1.00430931, ..., -0.04368215,
         1.75014627, -0.83369581]])

In [53]:
n_epoches = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1, 1), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name ="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
trainig_op = tf.assign(theta, theta - learning_rate * gradients)

In [56]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoches):
        if epoch % 100 ==0:
            print('epoch', epoch, "MSE = ", mse.eval())
        sess.run(trainig_op)
    best_theta = theta.eval()    

epoch 0 MSE =  14.077377
epoch 100 MSE =  0.84600574
epoch 200 MSE =  0.62776566
epoch 300 MSE =  0.597825
epoch 400 MSE =  0.57821393
epoch 500 MSE =  0.5639559
epoch 600 MSE =  0.55354536
epoch 700 MSE =  0.54593
epoch 800 MSE =  0.5403484
epoch 900 MSE =  0.5362497


In [57]:
best_theta

array([[ 2.0685525 ],
       [ 0.8355362 ],
       [ 0.15187904],
       [-0.21682015],
       [ 0.23908356],
       [ 0.00747133],
       [-0.04225472],
       [-0.6528395 ],
       [-0.6212864 ]], dtype=float32)

In [ ]:
# using autodiff to calculate the partial derviatives fro you

n_epoches = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1, 1), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name ="mse")
gradients = tf.gradients(mse, [theta])[0]  # the only change
trainig_op = tf.assign(theta, theta - learning_rate * gradients)

In [58]:
# using built in gradient decent
n_epoches = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1, 1), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name ="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate) # the cahnge
trainig_op = optimizer.minimize(mse) # the change

In [70]:
# Mini batch gradient descent
tf.reset_default_graph()

n_epoches = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None,n+1),name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1, 1), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name ="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate) # the cahnge
trainig_op = optimizer.minimize(mse) # the change

In [71]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [75]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoches):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index,batch_size)
            sess.run(trainig_op, feed_dict={X: X_batch, y:y_batch})
        best_theta = theta.eval()    

In [76]:
# visulaizing learning curves using Tensorboard
#adding time stamp to log dir to avoid messing the runs and hte visualizations
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run={}".format(root_logdir, now)

In [78]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [80]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoches):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index,batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                steps = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, steps)
            sess.run(trainig_op, feed_dict={X: X_batch, y:y_batch})
        best_theta = theta.eval()   

In [81]:
file_writer.close()